# problem statement:
What is the best font to use for my newsletter?




In [18]:
import requests
import pandas
from bs4 import BeautifulSoup
import re

In [2]:
# company_website_pairs = []
# with open('tech_websites.csv', newline='') as csvfile:
#     tech_websites = csv.reader(csvfile, dialect='excel')
#     for row in tech_websites:
#         # print(', '.join(row))
#         # name, website = row.split(',')
#         name, website = row[0], row[1]
#         company_website_pairs.append((name, website))
# # company_website_pairs

df = pandas.read_csv('tech_websites.csv', index_col=False)
df.head()

,company,https://website
0,youtube,https://youtube.com
1,google,https://google.com
2,gmail,https://gmail.com
3,google drive,https://drive.google.com
4,spotify,https://spotify.com


In [3]:
# a = df['company'] == 'apple'
# df[a]
apple_df = df[df['company'] == 'apple']
URL = apple_df.iat[0,1] # :(


page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")

font_tags = []
results = soup.find_all("link")
for result in results:
    # print(result.prettify())
    result_pretty = result.prettify()
    if "font" in result_pretty:
        font_tags.append(result_pretty)

# find 'fonts?families=' parse until "
sub = 'fonts?families='
families = "" 
for tag in font_tags:
    ind = tag.find(sub) + len(sub)

    for i in range(ind, len(tag)):
        if tag[i] == "\"":
            families = tag[ind:i]
            break
        else:
            i+=1
families


'SF+Pro,v3|SF+Pro+Icons,v3'

In [19]:
# download youtube html
youtube_df = df[df['company'] == 'youtube']
URL = youtube_df.iat[0,1] # :(
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())
# html_tag = soup.find("html")
# print(html_tag)
style_string = soup.html['style']
print(style_string)

# extract font families
sub_string = "font-family:"
families = "" 
index = style_string.find(sub_string)
end_index = index + len(sub_string) -1
# print(style_string[index])
# print(style_string[end_index])
sub_string_end = ";"
fam_index = style_string.find(sub_string_end, end_index )
# print(fam_index)
# print(style_string[fam_index])
families = style_string[end_index+1: fam_index]
# print(families)
# x = re.search("^font-family: .*;$", style_string)
# print(x)
font_families = families.strip().split(',')
print(font_families)

font-size: 10px;font-family: Roboto, Arial, sans-serif;


In [38]:
bad = []
for ind, row in df.iterrows():
    # print(row)
    company = row[0]
    URL = row[1]
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    try:
        style_string = soup.html['style']
        # extract font families
        sub_string = "font-family:"
        families = "" 
        index = style_string.find(sub_string)
        end_index = index + len(sub_string) -1
        sub_string_end = ";"
        fam_index = style_string.find(sub_string_end, end_index )
        families = style_string[end_index+1: fam_index]
        font_families = families.strip().split(',')
        print(font_families)
    except:
        bad.append(company)
print("------------------------")
print(bad)

['Roboto', ' Arial', ' sans-serif']
['color: rgb(var(--ig-secondary-background)']
------------------------
['google', 'gmail', 'google drive', 'spotify', 'twitter', 'microsoft', 'amazon', 'facebook', 'messenger', 'whatsapp', 'twitch', 'tiktok', 'apple', 'app store', 'zoom', 'cash app', 'netflix', 'discord', 'shopify', 'uber', 'paypal']


In [32]:
# 
for row in df:
    URL = row.iat[0,1] # :(

youtube_df = df[df['company'] == 'youtube']
URL = youtube_df.iat[0,1] # :(
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
# print(soup.prettify())
# html_tag = soup.find("html")
# print(html_tag)


f
:
54
;
 Roboto, Arial, sans-serif
['Roboto', ' Arial', ' sans-serif']
